**Análise Exploratória dos Dados**

In [ ]:
#Importando as Bibliotecas Necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Para visualização de datas
import datetime as dt

# Para análise de clustering
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

Carregando os Dados

In [ ]:
df = pd.read_csv('/content/data.csv', encoding='CP1252')

Análise Exploratória Inicial

In [ ]:
# Visualizando as primeiras linhas
df.head(10)

In [ ]:
# Verificando os tipos de dados
df.info()

In [ ]:
# Resumo estatístico
df.describe()

In [ ]:
# Verificando valores nulos
df.isnull().sum()

In [ ]:
#Analisando a distribuição das variáveis
# Histogramas
df.hist(figsize=(10, 8))
plt.show()

# Boxplots
sns.boxplot(data=df[['Quantity', 'UnitPrice']])
plt.show()

**Analisando as Métricas RFV**

In [ ]:
#Calculando as métricas
# Certifique-se que 'InvoiceDate' seja do tipo datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Criando a coluna 'TotalPrice'
if 'TotalPrice' not in df.columns:
    df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Agrupar por CustomerID e calcular as métricas
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (df['InvoiceDate'].max() - x.max()).days,
    'InvoiceNo': lambda x: len(x),
    'TotalPrice': lambda x: x.sum()
})

rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalPrice': 'Monetary'
}, inplace=True)

Analisando a distribuição das métricas

In [ ]:
# Histogramas para Recency, Frequency e Monetary
sns.histplot(rfm['Recency'])
sns.histplot(rfm['Frequency'])
sns.histplot(rfm['Monetary'])
plt.show()

**Pré-processamento dos Dados: Preparando os Dados para a Análise de Clustering**

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

# Escolhendo o scaler (por exemplo, MinMaxScaler)
scaler = MinMaxScaler()

# Selecionando as features numéricas a serem normalizadas
numeric_features = ['Recency', 'Frequency', 'Monetary']
rfm_scaled = scaler.fit_transform(rfm[numeric_features])

Seleção de Features

In [ ]:
# Calculando a matriz de correlação
correlation_matrix = rfm.corr()
sns.heatmap(correlation_matrix, annot=True)
plt.show()

**Escolha do Algoritmo de Clustering**

In [ ]:
from sklearn.cluster import KMeans

# Assumindo que rfm_scaled contém os dados normalizados
kmeans = KMeans(n_clusters=3, random_state=42)  # Definindo 3 clusters
kmeans.fit(rfm_scaled)

In [ ]:
# Obter os rótulos dos clusters
labels = kmeans.labels_

In [ ]:
# Adicionar os rótulos ao DataFrame original
rfm['Cluster'] = labels

Analisar os clusters

In [ ]:
# Visualizar os clusters
sns.scatterplot(x='Recency', y='Monetary', hue='Cluster', data=rfm)
plt.title('Clusters de Clientes')
plt.show()

In [ ]:
# Calcular estatísticas descritivas para cada cluster
rfm.groupby('Cluster').agg({'Recency': 'mean', 'Frequency': 'mean', 'Monetary': 'mean'})

**Determinando o Número Ótimo de Clusters (K)**

In [ ]:
# Método do Cotovelo
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42) # Indentado com 4 espaços
    kmeans.fit(rfm_scaled) # Indentado com 4 espaços
    wcss.append(kmeans.inertia_) # Indentado com 4 espaços

plt.plot(range(1, 11), wcss)
plt.title('Método do Cotovelo')
plt.xlabel('Número de clusters')
plt.ylabel('WCSS')  # Within-Cluster Sum of Squares
plt.show()

In [ ]:
# Silhouette Score:
from sklearn.metrics import silhouette_score

silhouette_avg = silhouette_score(rfm_scaled, labels)
print("Silhouette score:", silhouette_avg)

**Análise dos Clusters: Desvendando os Segmentos de Clientes**

Descrição dos Clusters

In [ ]:
#Estatísticas Descritivas
rfm.groupby('Cluster').agg({'Recency': ['mean', 'median'],
                           'Frequency': ['mean', 'median'],
                           'Monetary': ['mean', 'median', 'sum']})

In [ ]:
# Boxplots
sns.boxplot(x='Cluster', y='Monetary', data=rfm)
plt.title('Comparação do Valor de Compra por Cluster')
plt.show()

In [ ]:
# Função para criar histogramas para cada variável em cada cluster
def plot_histograms(df, numeric_columns, cluster_column='Cluster'):
    for column in numeric_columns:
        sns.histplot(data=df, x=column, hue=cluster_column, kde=True)
        plt.title(f'Distribuição de {column} por Cluster')
        plt.show()

# Exemplo de uso:
numeric_columns = ['Recency', 'Frequency', 'Monetary']
plot_histograms(rfm, numeric_columns)

In [ ]:
#Gráficos de Barras
# Contando a quantidade de clientes em cada cluster
cluster_counts = rfm['Cluster'].value_counts()

# Criando o gráfico de barras
sns.countplot(x='Cluster', data=rfm)
plt.title('Distribuição dos Clientes por Cluster')
plt.ylabel('Número de Clientes')
plt.show()

**Insights e Recomendações Baseadas nos Clusters**


**Cluster 0: Os Fiéis:** Clientes com alta frequência de compra e alto valor médio por pedido.

**Cluster 1: Os Ocasionais:** Clientes com baixa frequência de compra e valor médio por pedido moderado.

**Cluster 2:** Os Exploradores: Clientes com alta frequência de compra, mas baixo valor médio por pedido.

**Recomendações:**

**Cluster 0: Os Fiéis:**
**Programas de Fidelidade:** Crie um programa de pontos ou benefícios exclusivos para recompensar a lealdade desses clientes.
**Produtos Personalizados:** Ofereça produtos ou serviços personalizados com base no histórico de compras.
Comunicação Exclusiva: Envie comunicados e promoções personalizadas para este grupo.

**Cluster 1: Os Ocasionais:**
**Campanhas de Reaproximação:** Utilize e-mails, SMS ou notificações push para lembrar esses clientes da marca e oferecer promoções especiais.
**Análise de Carrinho Abandonado:** Identifique os motivos pelos quais esses clientes abandonam o carrinho e ofereça incentivos para finalizar a compra.
Produtos Complementares: Sugira produtos complementares aos que o cliente já adquiriu.

**Cluster 2: Os Exploradores:**
**Produtos em Destaque:** Destaque produtos com maior margem de lucro ou novos lançamentos.
Kits e Pacotes: Crie kits ou pacotes com produtos complementares para aumentar o valor médio do pedido.
**Programas de Referência:** Incentive esses clientes a indicarem a marca para seus amigos e familiares.

**Outras Recomendações Gerais:**
**Segmentação de Marketing:** Utilize os clusters para criar campanhas de marketing mais direcionadas e personalizadas.

**Gerenciamento de Estoque:** Ajuste o estoque de acordo com a demanda de cada cluster.

**Desenvolvimento de Produtos:** Crie novos produtos ou serviços que atendam às necessidades específicas de cada cluster.

**Atendimento ao Cliente:** Ofereça um atendimento personalizado para cada cluster, com foco nas suas necessidades e expectativas.

**Como Implementar as Recomendações:**
**Ferramentas de CRM:** Utilize um sistema de CRM para armazenar as informações dos clientes e personalizar as ações de marketing.

**Plataformas de E-mail Marketing:** Envie e-mails segmentados com base nos clusters.

**Análise Web:** Utilize ferramentas de análise web para acompanhar o comportamento dos clientes em seu site e aplicativo.

**Testes A/B:** Faça testes A/B para comparar diferentes abordagens e identificar o que funciona melhor para cada cluster.